In [35]:
#Import dependencies
import quandl
import pandas as pd
import numpy as np 
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [36]:
#Retrieve data from Quandl
quandl.ApiConfig.api_key = "1sNXfWjF5c7B4CLEZZyT"

In [37]:
#Assign start and end dates form the stock
start = pd.to_datetime('2014-10-17')
end = pd.to_datetime('2019-10-17')
data_TGT = quandl.get('EOD/TGT', start_date='2014-10-17', end_date='2019-10-17')
data_TGT.tail()

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
2019-10-11,111.85,112.94,111.21,111.83,4581116.0,0.0,1.0,111.85,112.94,111.21,111.83,4581116.0
2019-10-14,111.85,112.23,110.95,111.15,2422448.0,0.0,1.0,111.85,112.23,110.95,111.15,2422448.0
2019-10-15,111.56,112.30,111.34,111.89,2717961.0,0.0,1.0,111.56,112.30,111.34,111.89,2717961.0
2019-10-16,111.79,112.21,111.18,112.19,2340192.0,0.0,1.0,111.79,112.21,111.18,112.19,2340192.0
2019-10-17,112.41,113.71,112.34,113.23,4115616.0,0.0,1.0,112.41,113.71,112.34,113.23,4115616.0


In [38]:
# Get the Adjusted Close Price 
data_TGT = data_TGT[['Adj_Close']] 
# Take a look at the new data 
print(data_TGT.head())

            Adj_Close
Date                 
2014-10-17  49.932150
2014-10-20  50.963422
2014-10-21  52.104583
2014-10-22  51.842539
2014-10-23  52.434252


In [39]:
# A variable for predicting 'n' days out into the future
forecast_out=30     
#Create another column (the target ) shifted 'n' units up
data_TGT['Prediction'] = data_TGT[['Adj_Close']].shift(-forecast_out)
#print the new data set
print(data_TGT.tail())


            Adj_Close  Prediction
Date                             
2019-10-11     111.83         NaN
2019-10-14     111.15         NaN
2019-10-15     111.89         NaN
2019-10-16     112.19         NaN
2019-10-17     113.23         NaN


In [40]:
### Create the independent data set (X)  #######
# Convert the dataframe to a numpy array
X = np.array(data_TGT.drop(['Prediction'],1))

#Remove the last '30' rows
X = X[:-forecast_out]
print(X)

[[ 49.93215   ]
 [ 50.96342176]
 [ 52.10458313]
 ...
 [106.71      ]
 [107.36      ]
 [108.54      ]]


In [41]:
### Create the dependent data set (y)  #####
# Convert the dataframe to a numpy array 
y = np.array(data_TGT['Prediction'])
# Get all of the y values except the last '30' rows
y = y[:-forecast_out]
print(y)

[ 61.97227609  62.24486892  62.46635059 ... 111.89       112.19
 113.23      ]


In [42]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [43]:
# Create and train the Support Vector Machine (Regressor) 
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1) 
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [44]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  0.6427129527418038


In [45]:
# Create and train the Linear Regression  Model
lr = LinearRegression()
# Train the model
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [46]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.6553818190063396


In [47]:
# Set x_forecast equal to the last 30 rows of the original data set from Adj. Close column
x_forecast = np.array(data_TGT.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

[[109.85]
 [107.95]
 [108.83]
 [108.97]
 [108.94]
 [107.99]
 [106.96]
 [107.28]
 [106.97]
 [107.87]
 [106.67]
 [108.78]
 [107.45]
 [106.8 ]
 [106.29]
 [106.12]
 [106.91]
 [105.94]
 [105.16]
 [106.62]
 [109.1 ]
 [107.95]
 [108.52]
 [110.36]
 [110.57]
 [111.83]
 [111.15]
 [111.89]
 [112.19]
 [113.23]]


In [48]:
# Print linear regression model predictions for the next '30' days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

[108.96719856 107.16295518 107.99860474 108.13154899 108.10306094
 107.20093925 106.22284941 106.52672198 106.23234543 107.08698703
 105.9474649  107.95112465 106.68815429 106.07091313 105.58661622
 105.42518392 106.17536932 105.25425559 104.5135662  105.89998481
 108.25499723 107.16295518 107.70422819 109.45149547 109.65091184
 110.84741009 110.20168088 110.9043862  111.18926673 112.17685258]


In [49]:
# Print support vector regressor model predictions for the next '30' days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[110.50488149 112.57181361 113.08365682 112.96262722 112.99434456
 112.62927494 110.99721189 111.47200615 111.01046278 112.45141102
 110.66919557 113.11063844 111.75427976 110.80222873 110.41882488
 110.36993637 110.93277291 110.3527248  110.41731189 110.6244232
 112.78743225 112.57181361 113.12421341 107.74098247 106.3598551
  96.48494501 102.01781853  95.99988104  93.62828558  86.68281704]
